# 1D Wave-equation / linearized shallow-water equation on a periodic domain

The one-dimensional shallow-water equations can be linearized and simplified to the 1D wave-equation:

$\partial h/\partial t + D \partial u / \partial x = 0$

$\partial u/\partial t + g \partial h / \partial x = 0$

where $h$ denotes the water-height above the reference, $u$ the velocity, $D$ the depth below the reference and $t,x$ time and space.

In [ ]:
# goto folder of this notebook (If you move this notebook then also copy the Project.toml file and run instantiate to install the packages)
cd(@__DIR__)
# load the environment
using Pkg 
Pkg.activate(".")  # activate the environment for this notebook
# load the packages
using OrdinaryDiffEq
using ComponentArrays
using Plots
using JLD2


In [ ]:
# define the model

"""
    Wave1DPeriodic
    container for the parameters of the 1D wave equation
    We use a 'functor', i.e. a struct that behaves like a function.
    So this function can contain data, eg parameters.
"""
struct Wave1DPeriodic_cpu
    g::Float64 # gravity
    D::Float64 # depth
    L::Float64 # length
    dx::Float64 # spatial step
    nx::Int64 # number of spatial points
end

"""
    initial state for the 1D wave equation
    We use ComponentArrays to store the state variables
    x.h : height
    x.u : velocity
    The data is stored on a regular but staggered grid
    h at: 0.0, dx, 2dx, ...
    u at: dx/2, 3dx/2, 5dx/2, ...

    The initial state is a Gaussian bump in the height field
    of height h0, centered at x_center with a width of w times the length
    and located at c times the length.
"""
function initial_state_bump(f::Wave1DPeriodic_cpu, h0=1.0, w=0.05, c=0.5)
    x_center=f.L*c
    width=f.L*w
    x_h = 0.0:f.dx:(f.L-f.dx/2)
    h = h0.*exp.(-((x_h .- x_center).^2) ./ (2 * width^2))
    u = zeros(f.nx) .+ 0.0
    x = ComponentVector(h=h,u=u)
    return x
end

"""
   Compute the spatial derivative of h
   function dh_dx!(∂h∂x,h,dx)
"""
function dh_dx!(∂h∂x,h,dx)
    nx=length(h)
    for i in 1:(nx-1)
        ∂h∂x[i] = (h[i+1]-h[i])/(dx)
    end
    ∂h∂x[end] = (h[1]-h[end])/dx
end

"""
   Compute the spatial derivative of u
   function du_dx!(∂u∂x,u,dx)
"""
function du_dx!(∂u∂x,u,dx)
    nx=length(u)
    for i in 2:nx
        ∂u∂x[i] = (u[i]-u[i-1])/(dx)
    end
    ∂u∂x[1] = (u[1]-u[end])/dx
end

"""
    Wave1DPeriodic
    Compute time derivative of the state
"""
function (f::Wave1DPeriodic_cpu)(dx_dt,x,p,t)
    dx=f.dx
    g=f.g
    D=f.D
    # temporary variables
    ∂h∂x = similar(x.h) # allocating version, is not optimal for performance
    ∂u∂x = similar(x.u)
    # compute spatial derivatives
    dh_dx!(∂h∂x,x.h,dx)
    du_dx!(∂u∂x,x.u,dx)
    # compute time derivatives
    @. dx_dt.u = -g * ∂h∂x
    @. dx_dt.h = -D * ∂u∂x
end



In [ ]:
# setting up the problem
# parameters
nx=200     # number of spatial points
L=10000.0  # length of the domain
dx=L/nx    # spatial step
D=10.0     # depth of the water
g=10.0     # acceleration due to gravity

# initial state
f=Wave1DPeriodic_cpu(g,D,L,dx,nx)
x0=initial_state_bump(f,1.0,0.05,0.5)

# time span
t_end=1000.0

# spatial grid (for plotting)
x_h = 0.0:f.dx:(f.L-f.dx/2)
x_u = (f.dx/2):f.dx:f.L
x=ComponentVector(h=x_h,u=x_u)

In [ ]:
# plot the initial state
p1=plot(x_h, x0.h, label="h(t=0)", xlabel="x", ylabel="h")
p2=plot(x_u, x0.u, label="u(t=0)", xlabel="x", ylabel="u")
plot(p1, p2, layout=(2,1), size=(800,600), title="Initial state of the 1D wave equation")

In [ ]:
prob = ODEProblem(f, x0, (0.0, t_end))
@time sol = solve(prob, Tsit5(),saveat=10.0) # with timing, but ouput only at t=10.0
sol = solve(prob, Tsit5()) # store output, no timing

nothing

In [ ]:
# create a movie of the solution
@time begin
    # create a movie of the solution
    anim = @animate for i in 1:length(sol.t)
        p1 = plot(x_h, sol[i].h, label="h(t=$(round(sol.t[i],digits=1))", xlabel="x", ylabel="h", ylim=(-2, 2))
        p2 = plot(x_u, sol[i].u, label="u(t=$(round(sol.t[i],digits=1))", xlabel="x", ylabel="u", ylim=(-2, 2))
        plot(p1, p2, layout=(2,1), size=(800,600), title="Solution of the 1D wave equation")
    end
    gif(anim, "wave1d_periodic.gif", fps=30)
end


In [ ]:
# save the solution to a JLD2 file
cfl_out=1.0 # CFL number for output
dt_out=cfl_out*f.dx/(sqrt(f.D*f.g)) # time step for output, based on CFL condition
times=collect(0.0:dt_out:t_end) # output times
solution=[]
for t in times
    push!(solution, sol(t))
end
# save the solution to a JLD2 file
save("wave1d_periodic.jld2", 
    "solution", solution,
    "times", times,
    "params", (g=f.g, D=f.D, L=f.L, dx=f.dx, nx=f.nx),
    "grid", x,
    "cfl_out", cfl_out,
    "dt_out", dt_out,
    "description", "Solution of the 1D wave equation with periodic boundary conditions, using a Gaussian bump as initial condition.")


In [ ]:
# load the solution from the JLD2 file for testing
ff=load("wave1d_periodic.jld2")

@show ff["solution"][1].h[1:20:end] # show some values of the initial height field